In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import sys
sys.path.append('/Users/wastechs/Documents/git-repos/energy-efficiency')
from lib.util.helper import query_table, weekday_time_series
import seaborn as sns
import plotly.express as px
from prophet import Prophet
from prophet.diagnostics import cross_validation
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
import plotly
from prophet.plot import plot_plotly, plot_components_plotly
from prophet.plot import add_changepoints_to_plot

In [ ]:
druck = weekday_time_series(sensor_id='xl106_druckmaschine_5T')
druck['kw'] = round(druck['kw'], 2)
druck['kw'] = druck['kw'].apply(lambda x: 0.0 if x == -0.0 else x)

In [ ]:
plotly.offline.init_notebook_mode()
px.line(
    x=druck.index, y=druck.kw, 
    title='Xl106 Druckmaschine - Sampled at 5 minute intervals',
    labels={
        'x': 'Time',
        'y': 'kW'
    },
    markers=False)

### Generalized Additive Model with Facebook's Prophet

"A fancy name for the **summation** of the outputs of different models"

$y(t) = g(t) + s(t) + h(t) + \epsilon_t$

where

$y(t) = $ output at time $t$

$g(t) = $ models non-periodic changes (growth)

$s(t) = $ models periodic changes using Fourier basis functions

$h(t) = $ holidays or other special "events"

### Piecewise Linear Model $g(t)$

Since the machine time series does not exhibit an increasing or decreasing growth / trend, a piecewise constant rate of growth is suitable:

 $g(t) = (k + a(t)^T\delta)t + (m + a(t)^T\gamma)$

 where


$k = $ growth rate

$\delta = $ rate adjustments

$m = $ offset parameter

$\gamma_j = $ is set to $-s_j\delta_j$ to make the function continuous


### Diving into $g(t)$

$g(t) = (k + a(t)^T\delta)t$

The growth model consists of a base trend $k$ and preset changepoints at which the growth rate can be adjusted by $s_j$
 - These preset changepoints are defined in an $S$ vector with changepoints at times $s_j = 1, . . .,S$

At each unique changepoint $s_j$, the growth rate is **adjusted** by $\delta_j$
 - Thus, we can define all growth rate adjustments by the vector $\delta$

Summary - The growth rate is adjusted every time step that $t$ surpasses a changepoint $s_j$. The growth rate then becomes the base rate plus the sum of all adjustments up to that point:

$k + \sum_{j:t > s_j}\delta_j$

### Specifying Changepoints and Their Scale

The changepoints $s_j$ can be specified by the analyst using known dates / times of machine operation, product launches and other growth / trend altering events
 - Or, it may be automatically selected given a set of candidates (namely, by specifying a distribution)

If we want to specify a large number of changepoints, i.e., machine start up, shut down, standy by, etc. then we can specify a prior distribution on $\delta$:

$\delta$ ~ $Laplace(0, \tau)$

where

$\tau = $ directly controls the flexibility of the model in altering its rate (higher $\tau$ values make the model follow the variance better, but increase overfitting)

### Seasonality With Fourier Basis Functions

A Fourier sum can approximate **any** arbitrary periodic signal:

$s(t) = \sum^N_{n=1} \left[a_n \text{cos}\left(\frac{2 \pi nt}{P} \right) + b_n \text{sin}\left(\frac{2 \pi nt}{P}\right) \right]$

The number of terms in the partial sum (the order parameter in Prophet) determines how quickly the seasonality can change and the period $P$ is the length of the cycle, i.e., 365.25 for yearly data or 7 for weekly data)
 - Higher order values indicate higher frequency changes

Fourier Basis functions are a collection of $sine$ and $cosine$ functions that can be used for approximating arbitrary smooth seasonal effects 
 - _The Idea_: We can capture the sesonal effects of the machines (hour of day, weekly, . . .) with the fourier basis functinos



### Summarizing

The output $y$ at time $t$ is a function of:

$g(t)$ = which is the growth at time $t$ which itself is conditional on whether $t$ is $>$ a changepoint $s_j$ and the degree of the effect of the changepoint is determined by $\delta$ which can be specified a priori

$s(t)$ = which is pre-defined seasonality components based on domain knowledge, i.e., hourly and weekly seasonality of electricity consumption typically conditional on production schedules and human behavior

$h(t) = $ Swiss holidays or Gassmann special days (yet to be defined)

Positive skew in the variance of the time series could be a problem as GAMs assume Gaussian noise of the residuals
 - How to handle when the machine is off? (0 values)

In [ ]:
time = pd.Timestamp('2021-10-11')
end_time = pd.Timestamp('2021-10-15')
druck[(druck.index.date >= time) & (druck.index.date <= end_time)]

In [ ]:
train_druck = druck[druck.index.day <= 14]
test_druck = druck[druck.index.day >= 15]
train_druck.tail()

In [ ]:
test_druck.head()

In [ ]:
train_druck = pd.DataFrame(data=train_druck['kw'], index=train_druck.index)
test_druck = pd.DataFrame(data=test_druck['kw'], index=test_druck.index)

train_druck.reset_index(inplace=True)
test_druck.reset_index(inplace=True)

# Standardizing helps with convergence
scale = StandardScaler()

train_druck['kw'] = scale.fit_transform(train_druck['kw'].values.reshape(-1, 1))

scale.fit(train_druck['kw'].values.reshape(-1, 1))
test_druck['kw'] =  scale.transform(test_druck['kw'].values.reshape(-1, 1))

In [ ]:
# Prophet Naming Conventions
train_druck = train_druck.rename(columns={'t': 'ds', 'kw': 'y'})
test_druck = test_druck.rename(columns={'t': 'ds', 'kw': 'y'})

In [ ]:
pd.infer_freq(train_druck.ds)

In [ ]:
train_druck['ds'].min()

In [ ]:
# Training dataframe is ready
train_druck

In [ ]:
tau_prior = np.arange(0.05, 20, 0.05)
tau_prior

In [ ]:
def build_prophet_model(df, future, changepoint_prior_scale=0.05, other_seasonality=None):
    
    # Initializing the model
    m = Prophet(
        changepoint_prior_scale=changepoint_prior_scale,
        growth='linear'
        #seasonality_mode='multiplicative'
    )
    
    # Add additional seasonality such as "hourly" seasonality
    # Higher fourier order captures how "quickly" the seasonality can change
    m.add_seasonality(name='hourly', period=0.041, fourier_order=30)

    m.fit(df)
    forecast = m.predict(future)

    return m, forecast


In [ ]:
# For forecasting - build a dataframe with frequency and length of time to forecast
future = pd.DataFrame(
    data=pd.date_range('2021-10-11 00:00:00', '2021-10-15 23:55:00', freq='5T'),
    columns=['ds']
    )

In [55]:
# Running the model
model, predictions = build_prophet_model(train_druck, future, 0.8, None)

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.


Initial log joint probability = -153.606
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       1261.54     0.0600397        30.983           1           1      122   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       1268.94       0.14614       27.7546           1           1      237   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       1275.04     0.0128378       21.3134           1           1      351   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399        1276.8    0.00273899       18.9906      0.8598      0.8598      469   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       1282.37      0.011732       35.9639           1           1      583   
    Iter      log prob        ||dx||      ||grad||       alpha  

In [56]:
full_df = pd.concat([train_druck, test_druck])
len(full_df)

1440

In [ ]:
len(predictions)

In [57]:
full_df.reset_index(inplace=True)

In [ ]:
predictions

### Predictions

_For observation #500_

$\hat{y} = g(t) + s(t)$ = **0.763812**

$g(2021-10-12 \space 17:40:00) = 0.556047$

$s(2021-10-12 \space 17:40:00) = 0.207765$

In [59]:
# Clamp predictions below 0 kw to 0
#predictions['yhat'] = predictions['yhat'].apply(lambda x: -1.0 if x <= 0 else x)
coef = predictions[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
coef['actual'] = full_df['y']

/var/folders/5n/lzw120x534d6n5nbhk8qwzjr0000gs/T/ipykernel_12420/3656225224.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [60]:
coef

,ds,yhat,yhat_lower,yhat_upper,actual
0,2021-10-11 00:00:00,-1.064864,-1.731025,-0.401589,-1.003311e+00
1,2021-10-11 00:05:00,-1.119374,-1.808972,-0.502411,-1.003311e+00
2,2021-10-11 00:10:00,-1.115581,-1.774817,-0.507992,-1.003311e+00
3,2021-10-11 00:15:00,-1.130289,-1.848114,-0.440091,-1.003311e+00
4,2021-10-11 00:20:00,-1.034134,-1.695917,-0.376528,-1.003311e+00
...,...,...,...,...,...
1435,2021-10-15 23:35:00,-0.507794,-3.951225,3.117613,2.467162e-17
1436,2021-10-15 23:40:00,-0.565426,-4.042371,3.069882,2.467162e-17
1437,2021-10-15 23:45:00,-0.548686,-3.879978,3.092577,2.467162e-17
1438,2021-10-15 23:50:00,-0.542532,-4.061129,3.050827,2.467162e-17


In [61]:
test_all = coef[coef['ds'] >= '2021-10-15']
truth = test_all['actual']
preds = test_all['yhat']

rmse = mean_squared_error(truth, preds)
mape = mean_absolute_percentage_error(truth, preds)
rmse, mape

(4.56895107635984, 1876984565988823.2)

In [46]:
#mape = mean_absolute_percentage_error(coef['actual'], coef['yhat'])
rmse = mean_squared_error(coef['actual'], coef['yhat'])
rmse

1.1850670436279975

In [ ]:
# Get actual values outside of 95% CI
oob_upper = np.array(np.where(coef['actual'] > coef['yhat_upper'])).flatten()
oob_lower = np.array(np.where(coef['actual'] < coef['yhat_lower'])).flatten()

oob_upper = coef.iloc[oob_upper]
oob_lower = coef.iloc[oob_lower]

In [ ]:
# Forecasted values seasonal components
model.plot_components(predictions)

In [ ]:
upper = oob_upper['actual'].to_numpy()
lower = oob_lower['actual'].to_numpy()
upper

In [ ]:
fig = plot_plotly(model, predictions, uncertainty=True)
fig.show()

In [ ]:
fig = model.plot(predictions)

In [ ]:
# We can also plot the changepoints
a = add_changepoints_to_plot(fig.gca(), model, predictions)
fig

### Deploying Models with Prophet

Flow:

1.) Train a "good" first model :)

2.) Save model to json
   - _Note: The json file will be portable across systems, and deserialization is backwards compatible with older versions of prophet._

3.) As new data comes "in", pass the parameters from the saved model into the fitting for the next with the `kwarg` `init`

In [ ]:
# Saving fitted models
import json
from prophet.serialize import model_to_json, model_from_json

with open('serialized_model.json', 'w') as fout:
    json.dump(model_to_json(m), fout)  # Save fitted model

with open('serialized_model.json', 'r') as fin:
    m = model_from_json(json.load(fin))  # Load fitted model

In [ ]:
# Updating fitted models as new data comes in
def stan_init(m):
    """Retrieve parameters from a trained model.
    
    Retrieve parameters from a trained model in the format
    used to initialize a new Stan model.
    
    Parameters
    ----------
    m: A trained model of the Prophet class.
    
    Returns
    -------
    A Dictionary containing retrieved parameters of m.
    
    """
    res = {}
    for pname in ['k', 'm', 'sigma_obs']:
        res[pname] = m.params[pname][0][0]
    for pname in ['delta', 'beta']:
        res[pname] = m.params[pname][0]
    
    return res

In [ ]:
df = '<all the new data here in Prophet format>'
train_df = '<subset df into train and test>'
model = Prophet().fit(df)

updated_model = Prophet().fit(train_df, init=stan_init(model))